### Viewshed analysis with GRASS GIS

Identitfy areas visible from any point in obspoints

In [ ]:
"""
GRASS GIS example
"""


obspoints = '/home/gisuser/cwgis/demo2grass/obspoints.shp'
mdtraster = '/home/gisuser/cwgis/demo2grass/mdt.tif'
outvis    = '/home/gisuser/cwgis/results/demo2grass.tif'

In [ ]:
import geopandas as gp
import os

from glass.wenv.grs import run_grass

In [ ]:
# Create GRASS GIS Location
workspace = os.path.dirname(outvis)
location  = 'loc_' + os.path.splitext(os.path.basename(outvis))[0]

gbase = run_grass(workspace, location=location, srs=mdtraster)

# Start GRASS GIS Session

import grass.script.setup as gsetup
gsetup.init(gbase, workspace, location, 'PERMANENT')

In [ ]:
def rst_to_grs(rst, grsRst):
    """
    Raster to GRASS GIS Raster
    """
    
    from grass.pygrass.modules import Module
        
    m = Module(
        "r.in.gdal", input=rst, output=grsRst, flags='o',
        overwrite=True, run_=False, quiet=True,
    )
        
    m()
    
    return grsRst

In [ ]:
def grs_viewshed(dem, obs_pnt, out_rst, max_dist=None, obs_elv=None):
    """
    Compute viewshed
    """
    
    from grass.pygrass.modules import Module
    
    vshd = Module(
        "r.viewshed", input=dem, output=out_rst,
        coordinates=obs_pnt,
        flags="b", overwrite=True, run_=False, quiet=True,
        max_distance=-1 if not max_dist else max_dist,
        observer_elevation=1.75 if not obs_elv else obs_elv
    )
    
    vshd()
    
    return out_rst

In [ ]:
def rstcalc(expression, output):
    """
    Basic Raster Calculator
    """
    
    from grass.pygrass.modules import Module
        
    rc = Module(
        'r.mapcalc',
        f'{output} = {expression}',
        overwrite=True, run_=False, quiet=True
    )
        
    rc()
    
    return output

In [ ]:
def grs_to_rst(grsRst, rst):
    """
    GRASS Raster to Raster
    """
    
    grass_formats = {
        '.tif': 'GTiff',
        '.img': 'HFA'
    }
    
    fn, ff = os.path.splitext(rst)
    
    from grass.pygrass.modules import Module
        
    m = Module(
        "r.out.gdal", input=grsRst, output=rst,
        format=grass_formats[ff], flags='c',
        createopt='TFW=YES',
        overwrite=True, run_=False, quiet=True
    )
        
    m()
    
    return rst

In [ ]:
# Add DEM to GRASS GIS workspace

grsdem = rst_to_grs(mdtraster, 'dem')

# Read Observation Points Shape
points = gp.read_file(obspoints)

In [ ]:
points

In [ ]:
visibilities = []
for idx, row in points.iterrows():
    # Produce viewshed raster
    vrst = grs_viewshed(
        grsdem, (row.geometry.x, row.geometry.y),
        f"visrst_{str(idx)}",
        max_dist=10000
    )
    visibilities.append(vrst)

In [ ]:
print(visibilities)
print(" + ".join(visibilities))

In [ ]:
name = visibilities[0]
for i in range(1, len(visibilities)):
    name = name + ' + ' + visibilities[i]
print(name)

In [ ]:
# Sum all visibility rasters
fraster = rstcalc(" + ".join(visibilities), 'joinvis')

In [ ]:
# Export result
grs_to_rst(fraster, outvis)

In [ ]:
# Export all temp files

for r in visibilities:
    grs_to_rst(r, os.path.join(workspace, r + '.tif'))